In [1]:
#in this step we unpack the features, get the agreements and then use the agreements for training
#for NRC, if there isn't any emotion identified, it is tagged as neutral. 
#if there are ties, the ties are noted but not used in the final analysis 

In [2]:
import pandas as pd
import glob, os

In [3]:
from tqdm import tqdm
tqdm.pandas()

In [4]:
os.chdir(r"D:\PACK\PhD\Study1\cleaned_dataset")
os.listdir()

['best_model_state.bin',
 'cache_dir',
 'df_amt_test.h5',
 'df_amt_train.h5',
 'df_ISOT_test.h5',
 'df_ISOT_train.h5',
 'df_kdd_test.h5',
 'df_kdd_train.h5',
 'df_Liar_test.h5',
 'df_Liar_train.h5',
 'df_stance_test.h5',
 'df_stance_train.h5',
 'outputs',
 'runs',
 'semeval2016_stance_best_model_state.bin',
 'test.h5',
 'test_NER.h5',
 'test_NER_emotion.h5',
 'test_NER_emotion_stance.h5',
 'test_NER_emotion_stance_agreement.h5',
 'train.h5',
 'train_NER.h5',
 'train_NER_emotion.h5',
 'train_NER_emotion_stance.h5',
 'train_NER_emotion_stance_agreement.h5',
 'val.h5',
 'val_NER.h5',
 'val_NER_emotion.h5',
 'val_NER_emotion_stance.h5',
 'val_NER_emotion_stance_agreement.h5']

In [5]:
train_df = pd.read_pickle('train_NER_emotion_stance.h5',compression='xz')

In [6]:
train_df

,index,text,label,target,target_label,sentences,joined_sentences,emotion,stance,NRC,text_target,semeval2016
0,2812,California has had a movement going for the la...,0,California,GPE,[California has had a movement going for the l...,California has had a movement going for the la...,"[[{'label': 'neutral', 'score': 0.605741798877...","[[{'label': 'LABEL_0', 'score': 0.787573456764...","{'sadness': 6, 'negative': 7, 'surprise': 2, '...",(California has had a movement going for the l...,"([[0.96880317, 0.0060505117, 0.025146294]], AG..."
0,2812,California has had a movement going for the la...,0,Donald Trump,PERSON,[This state has long been the leader of progre...,This state has long been the leader of progres...,"[[{'label': 'neutral', 'score': 0.946747601032...","[[{'label': 'LABEL_0', 'score': 0.709447622299...","{'anticipation': 2, 'positive': 6, 'trust': 4,...",(This state has long been the leader of progre...,"([[0.98019, 0.004925918, 0.014884095]], AGAINST)"
0,2812,California has had a movement going for the la...,0,the United States,GPE,[This state has long been the leader of progre...,This state has long been the leader of progres...,"[[{'label': 'neutral', 'score': 0.720751821994...","[[{'label': 'LABEL_0', 'score': 0.643858253955...","{'anticipation': 2, 'positive': 12, 'trust': 8...",(This state has long been the leader of progre...,"([[0.97322446, 0.010055587, 0.016719911]], AGA..."
0,2812,California has had a movement going for the la...,0,Trump,PERSON,[This state has long been the leader of progre...,This state has long been the leader of progres...,"[[{'label': 'neutral', 'score': 0.536643862724...","[[{'label': 'LABEL_0', 'score': 0.821133673191...","{'anticipation': 4, 'positive': 10, 'trust': 8...",(This state has long been the leader of progre...,"([[0.94352233, 0.009292855, 0.04718487]], AGAI..."
0,2812,California has had a movement going for the la...,0,America,GPE,[Thanks to the fact that the Trump administrat...,Thanks to the fact that the Trump administrati...,"[[{'label': 'surprise', 'score': 0.88948446512...","[[{'label': 'LABEL_0', 'score': 0.818992555141...","{'trust': 2, 'sadness': 3, 'negative': 2, 'joy...",(Thanks to the fact that the Trump administrat...,"([[0.8429724, 0.032944206, 0.12408345]], AGAINST)"
...,...,...,...,...,...,...,...,...,...,...,...,...
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Illinois Lottery,ORG,[The state will be suspended from participatin...,The state will be suspended from participating...,"[[{'label': 'sadness', 'score': 0.664979696273...","[[{'label': 'LABEL_2', 'score': 0.889088034629...","{'negative': 1, 'anticipation': 2, 'joy': 2, '...",(The state will be suspended from participatin...,"([[0.9486447, 0.028873317, 0.02248202]], AGAINST)"
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Jason Schaumburg,PERSON,[The state will be suspended from participatin...,The state will be suspended from participating...,"[[{'label': 'sadness', 'score': 0.664979696273...","[[{'label': 'LABEL_2', 'score': 0.889088034629...","{'negative': 1, 'anticipation': 2, 'joy': 2, '...",(The state will be suspended from participatin...,"([[0.981726, 0.008949072, 0.009324892]], AGAINST)"
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Gianna Urgo,PERSON,"[With no money to pay road contractors, Illino...","With no money to pay road contractors, Illinoi...","[[{'label': 'sadness', 'score': 0.831223368644...","[[{'label': 'LABEL_2', 'score': 0.901882171630...","{'anger': 1, 'anticipation': 3, 'joy': 2, 'pos...","(With no money to pay road contractors, Illino...","([[0.55472624, 0.27097735, 0.17429636]], AGAINST)"
40498,3180,CHICAGO (Reuters) - Illinois’ Republican gover...,1,Medicaid,ORG,[A U.S. judge has set a June 20 deadline for I...,A U.S. judge has set a June 20 deadline for Il...,"[[{'label': 'neutral', 'score': 0.787567675113...","[[{'label': 'LABEL_2', 'score': 0.913707435131...",{'anticipation': 1},(A U.S. judge has set a June 20 deadline 

In [7]:
#cleaning the emotion data

def _emotion(df):
    '''this version is more faster'''
    emotion = []
    df['emotion'] = df.emotion.apply(lambda x:x[0])
    #print(df.emotion.iloc[0])
    
    #display(_df)
    for i in tqdm(range(len(df))):
        _df = pd.DataFrame.from_dict(df.emotion.iloc[i]).transpose()
        _df.columns = _df.iloc[0]
        _df = _df.drop(_df.index[0]).reset_index()
        emotion.append(_df)
        
    emotion = pd.concat(emotion)
    emotion = emotion[['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']]
    emotion['emotion_result'] = emotion.columns[emotion.values.argmax(1)]
    return emotion.reset_index(drop=True)

#cleaning the stance data

def _stance(df):
    stance = []
    for i in tqdm(range(len(df))):
        _df = pd.DataFrame(df.stance.iloc[i][0]).transpose()
        _df.columns = _df.iloc[0]
        _df = _df.drop(_df.index[0]).reset_index()

        stance.append(_df)
    stance = pd.concat(stance)
    stance = stance[['LABEL_0','LABEL_1','LABEL_2']]
    stance.columns = ['AGAINST','FAVOR','NONE']
    stance['stance_result'] = stance.columns[stance.values.argmax(1)]
    return stance.reset_index(drop=True)


def _nrc(df):
    def max_find_or_neutral(row):
        #find max number and max item
        #for i in row:
            #print(i)
        #print(max(row))
        #print([i for i in row if i==max(row)])

        if max(row) == 0:
            return 'Neutral'

        max_vals = [i for i in row if i==max(row)]

        if len(max_vals) == 1:
            return row.idxmax()

        else:
            return 'tie'
    '''
    Testing
    # display(nrc.iloc[0])
    # print(max_find_or_neutral(nrc.iloc[0]))

    # display(nrc.iloc[98])
    # print(max_find_or_neutral(nrc.iloc[98]))
    '''
    nrc = []
    for i in tqdm(range(len(df))):
        _df = pd.DataFrame(df.NRC.iloc[i], index=[0])
        nrc.append(_df)
        
    nrc = pd.concat(nrc)
    nrc = nrc.fillna(0)
    nrc = nrc[['surprise','anger','fear','joy','disgust','sadness']]

    lst = []
    for i in range(len(nrc)):
        lst.append(max_find_or_neutral(nrc.iloc[i]))

    nrc['nrc_result'] = lst

    return nrc.reset_index(drop=True)

def _semeval(df):
    semeval2016 = []
    for i in tqdm(range(len(df))):
        _df = pd.DataFrame(df.semeval2016.iloc[i][0])
        #_df.columns = _df.iloc[0]
        #_df = _df.drop(_df.index[0]).reset_index()
        _df.columns = ['AGAINST','FAVOR','NONE']

        #_df['Max'] = _df.idxmax(axis=0)

        semeval2016.append(_df)
        
    semeval2016 = pd.concat(semeval2016)
    semeval2016['semeval2016_result'] = semeval2016.columns[semeval2016.values.argmax(1)]
    return semeval2016.reset_index(drop=True)



In [8]:
def _emotion2(df):
    '''this version is more faster?'''
    emotion = []
    df['emotion'] = df.emotion.apply(lambda x:x[0])
    #print(df.emotion.iloc[0])

    
    #display(_df)
    for i in tqdm(range(len(df))):
        _lst = sorted(df.emotion.iloc[i], key=lambda d: d['label'])
        _lst = [d['score'] for d in _lst]

        emotion.append(_lst)
        
    emotion = pd.DataFrame(emotion)
    emotion.columns = ['anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise']
    emotion['emotion_result'] = emotion.columns[emotion.values.argmax(1)]
    return emotion.reset_index(drop=True)

def _stance2(df):
    '''this version is more faster?'''
    stance = []
    df['stance'] = df.stance.apply(lambda x:x[0])
    #print(df.emotion.iloc[0])

    
    #display(_df)
    for i in tqdm(range(len(df))):
        _lst = sorted(df.stance.iloc[i], key=lambda d: d['label'])
        _lst = [d['score'] for d in _lst]

        stance.append(_lst)
        
    stance = pd.DataFrame(stance)
    stance.columns = ['AGAINST','FAVOR','NONE']
    stance['stance_result'] = stance.columns[stance.values.argmax(1)]
    return stance.reset_index(drop=True)

def _nrc2(df):
    def max_find_or_neutral(row):
        #find max number and max item
        #for i in row:
            #print(i)
        #print(max(row))
        #print([i for i in row if i==max(row)])

        if max(row) == 0:
            return 'Neutral'

        max_vals = [i for i in row if i==max(row)]

        if len(max_vals) == 1:
            return row.idxmax()

        else:
            return 'tie'
    '''
    Testing
    # display(nrc.iloc[0])
    # print(max_find_or_neutral(nrc.iloc[0]))

    # display(nrc.iloc[98])
    # print(max_find_or_neutral(nrc.iloc[98]))
    '''
    nrc = []
    for i in tqdm(range(len(df))):
        d = {'anger':0, 'disgust':0, 'fear':0, 'joy':0, 'sadness':0, 'surprise':0}
        x = train_df.NRC.iloc[i]
        d |= x      
        nrc.append(d)
        
    nrc = pd.DataFrame.from_dict(nrc)
    nrc = nrc.fillna(0)
    nrc = nrc[['surprise','anger','fear','joy','disgust','sadness']]

    lst = []
    for i in tqdm(range(len(nrc))):
        lst.append(max_find_or_neutral(nrc.iloc[i]))

    nrc['nrc_result'] = lst

    return nrc.reset_index(drop=True)



In [ ]:
%%timeit
_emotion(train_df[:1000])

In [ ]:
%%timeit
_emotion2(train_df[:1000])

In [ ]:
%%timeit
_stance(train_df[:1000])

In [ ]:
%%timeit
_stance2(train_df[:1000])

In [ ]:
%%timeit
_nrc(train_df[:1000])

In [ ]:
%%timeit
_nrc2(train_df[:1000])

In [ ]:
_nrc2(train_df[:1000])

In [ ]:
%%timeit
_semeval(train_df[:1000])

In [ ]:
emotion = _emotion2(train_df) #optimised
stance = _stance2(train_df) #optimised
nrc = _nrc2(train_df) #optimised
semeval = _semeval(train_df)

In [ ]:
df2 = pd.concat([stance.reset_index(drop=True),semeval.reset_index(drop=True),emotion.reset_index(drop=True),nrc.reset_index(drop=True)],axis=1)
df2

In [ ]:
df2.columns

In [ ]:
df2['agreed_stance'] = df2['stance_result']==df2['semeval2016_result']
print(df2.agreed_stance.value_counts())
df2['agreed_emotion'] = df2['emotion_result']==df2['nrc_result']
print(df2.agreed_emotion.value_counts())
train_df = train_df.join(df2, how='inner')


In [ ]:
train_df

In [ ]:
train_df.to_pickle('train_NER_emotion_stance_agreement.h5',compression='xz')


In [ ]:
test_df = pd.read_pickle('test_NER_emotion_stance.h5',compression='xz')

In [ ]:
test_df

In [ ]:
emotion = _emotion2(test_df)
stance = _stance2(test_df)
nrc = _nrc2(test_df)
semeval = _semeval(test_df)

In [ ]:
df2 = pd.concat([stance.reset_index(drop=True),semeval.reset_index(drop=True),emotion.reset_index(drop=True),nrc.reset_index(drop=True)],axis=1)
df2

In [ ]:
df2['agreed_stance'] = df2['stance_result']==df2['semeval2016_result']
print(df2.agreed_stance.value_counts())
df2['agreed_emotion'] = df2['emotion_result']==df2['nrc_result']
print(df2.agreed_emotion.value_counts())
test_df = test_df.join(df2, how='inner')
test_df

In [ ]:
test_df.to_pickle('test_NER_emotion_stance_agreement.h5',compression='xz')


In [ ]:
val_df = pd.read_pickle('val_NER_emotion_stance.h5',compression='xz')

In [ ]:
display(_emotion(val_df[:1000]))
display(_stance(val_df[:1000]))
display(_nrc(val_df[:1000]))
display(_semeval(val_df[:1000]))

In [ ]:
emotion = _emotion2(val_df)
stance = _stance2(val_df)
nrc = _nrc2(val_df)
semeval = _semeval(val_df)

In [ ]:
df2 = pd.concat([stance.reset_index(drop=True),semeval.reset_index(drop=True),emotion.reset_index(drop=True),nrc.reset_index(drop=True)],axis=1)
df2

In [ ]:
df2['agreed_stance'] = df2['stance_result']==df2['semeval2016_result']
print(df2.agreed_stance.value_counts())
df2['agreed_emotion'] = df2['emotion_result']==df2['nrc_result']
print(df2.agreed_emotion.value_counts())
val_df = val_df.join(df2, how='inner')
val_df

In [ ]:
val_df.to_pickle('val_NER_emotion_stance_agreement.h5',compression='xz')
